In [10]:
import pandas as pd
import csv
import sys
import requests
from lxml import html as htmlRenderer
csv.field_size_limit(sys.maxsize)

9223372036854775807

In [11]:
csv_file_in = "/home/cflores/cflores_workspace/comments-retriever/results/lavanguardia/lavanguardia-01012019-15092019-contents.csv"
csv_file_out = "/home/cflores/cflores_workspace/comments-retriever/results/lavanguardia/lavanguardia-01012019-15092019-contents-fixed.csv"

In [12]:
def is_valid_csv(csv_file):
    invalid_rows = []
    no_content = []
    no_title = []
    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        counter = 0
        for r in reader:
            if not r[0]:
                print(" row -{}- row has not url.".format(counter))
                invalid_rows.append(counter)
            if not r[1]: 
                print(" row -{}- row has not content.".format(counter))
                invalid_rows.append(counter)
                no_content.append(r[0])
            if not r[2] and r[2] == r[0]:
                print(" row -{}- row has not title.".format(counter))
                invalid_rows.append(counter)
                no_title.append(r[0])
            counter = counter + 1
        
    return len(list(dict.fromkeys(invalid_rows))) == 0, invalid_rows, no_content, no_title

In [13]:
valid, invalid_rows, no_content, no_title = is_valid_csv(csv_file_in)

 row -792- row has not content.
 row -2171- row has not content.
 row -4125- row has not content.
 row -5223- row has not content.
 row -6396- row has not content.


In [14]:
no_title

[]

In [20]:
valid, invalid_rows, no_content, no_title = is_valid_csv(csv_file_out)

 row -792- row has not content.
 row -2171- row has not content.
 row -4125- row has not content.
 row -5223- row has not content.
 row -6396- row has not content.


In [16]:
no_content

['https://www.lavanguardia.com/deportes/rcde-espanyol/20190128/4641274077/espanyol-fichaje-wu-lei-china.html',
 'https://www.lavanguardia.com/economia/20190311/46965636146/luca-de-meo-seat-electrico-martorell-fabrica.html',
 'https://www.lavanguardia.com/local/barcelona/20190514/462232803499/josep-bou-pp-elecciones-municipales-barcelona.html',
 'https://www.lavanguardia.com/politica/20190615/462867740181/los-politicos-independentistas-viven-en-un-mundo-virtual.html',
 'https://www.lavanguardia.com/local/barcelona/20190721/463617642960/orantes-us-open-franco-sinatra-las-vegas.html']

In [17]:
def extract_content(url=None):
    response = requests.get(url)
    renderedPage = htmlRenderer.fromstring(response.text)
    queries_xpath = ["//div[@class='content-structure ']//p",
                     "//div[@itemprop='articleBody']//p"]
    contentStr = ""
    for q in queries_xpath:
        commentsElList = renderedPage.xpath(q)
        if len(commentsElList) > 0:
            contentArr = []
            for p in commentsElList:
                contentArr.append(p.text_content())
            contentStr = "".join([parrafo for parrafo in contentArr])
            break
    if not contentStr:
        print("\t -> url has not content found {}".format(url))
    return contentStr

In [18]:
def get_title(url):
    queryXpath = "//article//h1"
    response = requests.get(url)
    renderedPage = htmlRenderer.fromstring(response.text)
    el = renderedPage.xpath(queryXpath)
    title = url
    if len(el) == 0:
        print(" -> title not found in: {}".format(url))
    else:
        title = el[0].text
    return title

In [19]:
keys = []
no_content = []
no_title = []
with open(csv_file_in, 'r') as f:
    with open(csv_file_out, 'w') as fout:
        writer = csv.writer(fout)
        reader = csv.reader(f)
        for r in reader:
            if not r[1]:
                # no content
                print(" => row with url {} has any content".format(r[0]))
                content = extract_content(r[0])
                if content:
                    print(" \t ===>>> content found ")
                r[1] = content
            else:
                r[1] = r[1].replace("\t", "").replace("\n", "").lstrip().rstrip()
            if not r[2] or r[2] == r[0]:
                # no title
                print(" => row with url {} has any title".format(r[0]))
                r[2] = get_title(r[0])
            
            writer.writerow(r)

 => row with url https://www.lavanguardia.com/lacontra/20190114/454116566660/quien-se-lo-pueda-permitir-no-tendra-coche-privado.html has any title
 -> title not found in: https://www.lavanguardia.com/lacontra/20190114/454116566660/quien-se-lo-pueda-permitir-no-tendra-coche-privado.html
 => row with url https://www.lavanguardia.com/deportes/rcde-espanyol/20190128/4641274077/espanyol-fichaje-wu-lei-china.html has any content
	 -> url has not content found https://www.lavanguardia.com/deportes/rcde-espanyol/20190128/4641274077/espanyol-fichaje-wu-lei-china.html
 => row with url https://www.lavanguardia.com/lacontra/20190130/4691956504/las-madres-deben-tributar-menos-y-tener-voto-cualificado.html has any title
 -> title not found in: https://www.lavanguardia.com/lacontra/20190130/4691956504/las-madres-deben-tributar-menos-y-tener-voto-cualificado.html
 => row with url https://www.lavanguardia.com/economia/20190311/46965636146/luca-de-meo-seat-electrico-martorell-fabrica.html has any conten